<a href="https://colab.research.google.com/github/abrazzaq02/ANN-DL/blob/main/NLP_LAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy gensim torch matplotlib scikit-learn nltk


In [ ]:
import numpy as np
from collections import defaultdict

corpus = [
    "king queen royal palace",
    "man woman boy girl",
    "paris france capital city",
    "rome italy capital city",
    "king man queen woman",
]

tokens = [w for line in corpus for w in line.split()]
vocab = sorted(set(tokens))
idx = {w:i for i,w in enumerate(vocab)}

window = 2
cooc = np.zeros((len(vocab), len(vocab)), dtype=np.float32)

for line in corpus:
    ws = line.split()
    for i, w in enumerate(ws):
        wi = idx[w]
        for j in range(max(0, i-window), min(len(ws), i+window+1)):
            if i == j: continue
            cooc[wi, idx[ws[j]]] += 1

def cosine(a,b):
    return a@b / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-9)

def most_sim(word, k=3):
    i = idx[word]
    sims = [(vocab[j], float(cosine(cooc[i], cooc[j]))) for j in range(len(vocab)) if j != i]
    return sorted(sims, key=lambda x: x[1], reverse=True)[:k]

print("Similar to 'king':", most_sim("city"))


Similar to 'king': [('paris', 0.8660253594734205), ('rome', 0.8660253594734205), ('france', 0.4714044890112376)]


In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

raw_text = """
Paris is the capital of France. Rome is the capital of Italy.
The king and the queen visited the royal palace.
A boy and a girl met a man and a woman in the city.
"""
sentences = [simple_preprocess(s) for s in raw_text.strip().split("\n")]

# Skip-gram (sg=1); negative sampling (negative>0)
model = Word2Vec(sentences,
                 vector_size=50, window=1, min_count=1,
                 sg=1, negative=5, epochs=200, workers=1, seed=0)

print(model.wv.most_similar("king", topn=5))
print(model.wv.similarity("paris", "france"))

[('visited', 0.5600429773330688), ('the', 0.3318120539188385), ('city', 0.27487456798553467), ('royal', 0.21467186510562897), ('girl', 0.12113984674215317)]
0.106571905


In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

raw_text = """
Paris is the capital of France. Rome is the capital of Italy.
The king and the queen visited the royal palace.
A boy and a girl met a man and a woman in the city.
"""
sentences = [simple_preprocess(s) for s in raw_text.strip().split("\n")]

# Skip-gram (sg=1); negative sampling (negative>0)
model = Word2Vec(sentences,
                 vector_size=50, window=2, min_count=1,
                 sg=1, negative=5, epochs=200, workers=1, seed=0)

print(model.wv.most_similar("king", topn=5))
print(model.wv.similarity("paris", "france"))

[('visited', 0.5827963352203369), ('the', 0.36465612053871155), ('city', 0.30424636602401733), ('royal', 0.24916714429855347), ('girl', 0.186227485537529)]
0.20128465
